# **Homework 2-1: Phoneme Classification**

* Slides: https://speech.ee.ntu.edu.tw/~hylee/ml/ml2021-course-data/hw/HW02/HW02.pdf
* Video (Chinese): https://youtu.be/PdjXnQbu2zo
* Video (English): https://youtu.be/ESRr-VCykBs

## **The DARPA TIMIT Acoustic-Phonetic Continuous Speech Corpus (TIMIT)**
The TIMIT corpus of reading speech has been designed to provide speech data for the acquisition of acoustic-phonetic knowledge and for the development and evaluation of automatic speech recognition systems.

This homework is a multiclass classification task, 
we are going to train a deep neural network classifier to predict the phonemes for each frame from the speech corpus TIMIT.

link: https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3

## **Download Data**
Download data from google drive, then unzip it.

You should have `timit_11/train_11.npy`, `timit_11/train_label_11.npy`, and `timit_11/test_11.npy` after running this block.<br><br>
`timit_11/`
- `train_11.npy`: training data<br>
- `train_label_11.npy`: training label<br>
- `test_11.npy`:  testing data<br><br>

**notes: if the google drive link is dead, you can download the data directly from Kaggle and upload it to the workspace**

因為kaggle有了,所以不用載~

In [ ]:
'''
!gdown --id '1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR' --output data.zip
!unzip data.zip
!ls 
'''

## **Preparing Data**
Load the training and testing data from the `.npy` file (NumPy array).

In [ ]:
import numpy as np

print('Loading data ...')

# data_root='./timit_11/'
data_root = '../input/ml2021spring-hw2/timit_11/timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

# In numpy, V.shape gives a tuple of ints of dimensions of V
# In pytorch, V.size() gives a size object
print('Size of training data: {}'.format(train.shape))
print('Size of training_label data: {}'.format(train_label.shape))
print('Size of testing data: {}'.format(test.shape))
'''
print('Size of training data: {}'.format(train.size)) # 所有 data量
print('Size of training_label data: {}'.format(train_label.size))
print('Size of testing data: {}'.format(test.size))
'''

# 我覺得需要研究一下裡面到底有什麼QQ
# 先取出第一個 11X39的data,代表11個frame,以及對應的39個 phoneum
# phoneum是不是就代表對應的相關程度??
# 看看正中間那個frame是不是真的和label標示的那個類別的關聯度最高
# print('train的第0項',train[0])
print(train_label[0])
# print('test的第0項',test[0])
train_center_frame = train[0].reshape(11,39)
train_center_frame = train_center_frame[5]
print('其中label對應的應該是正中間的那個frame, 所以是',train_center_frame)

所以就是總共有1229932筆 data, 每個 data都是(11,39)

* 11是frame的數量, 而 label是正中間那個frame
* 39是 MFCC的維度,代表 39種發音的one-hot vector

那麼 train_label 是之後才要幫忙取出來的意思? label直接就是那個 11frame中的正中間的frame的類別
助教投影片說 train_label是0-38的 發音label

## **Create Dataset**

In [ ]:
import torch
from torch.utils.data import Dataset

# 自己設定 dataset
class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)

Split the labeled data into a training set and a validation set, you can modify the variable `VAL_RATIO` to change the ratio of validation data.

In [ ]:
VAL_RATIO = 0.05

percent = int(train.shape[0] * (1 - VAL_RATIO))
# x表示 data, y表示label
# TODO? 要不要再 shuffle看看?
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Create a data loader from the dataset, feel free to tweak the variable `BATCH_SIZE` here.

In [ ]:
BATCH_SIZE = 64

from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y) # 丟進自己的 dataset去處理
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

Cleanup the unneeded variables to save memory.<br>

**notes: if you need to use these variables later, then you may remove this block or clean up unneeded variables later<br>the data size is quite huge, so be aware of memory usage in colab**

In [ ]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

## **Create Model**

Define model architecture, you are encouraged to change and experiment with the model architecture.

In [ ]:
import torch
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # TODO? 設一下dropout
        # self.Dropout = True # 怎麼好像沒用??
        self.layer1 = nn.Linear(429, 1024)
        self.layer2 = nn.Linear(1024, 512)
        self.layer3 = nn.Linear(512, 256)
        self.layer4 = nn.Linear(256, 128)
        self.layer5 = nn.Linear(128, 64)
        self.out = nn.Linear(64, 39) 
        # TODO? 要加上 Batch normalization的 layer
        
        self.batchNorm1 = nn.BatchNorm1d(1024)
        self.batchNorm2 = nn.BatchNorm1d(512)
        self.batchNorm3 = nn.BatchNorm1d(256)
        self.batchNorm4 = nn.BatchNorm1d(128)
        self.batchNorm5 = nn.BatchNorm1d(64)
        # self.batchNorm = nn.BatchNorm1d(11)

        self.act_fn = nn.Sigmoid()

    def forward(self, x):
        # TODO? 加一下 Batch normalization
        # 每次input是 batchsize x 11 x 39
        # batch_size =  int(x.size)//429
        '''
        x_bn = x.reshape(-1,11,39)
        x = self.batchNorm(x_bn)
        x = x.reshape(-1,429)
        '''
        x = self.layer1(x)
        x = self.batchNorm1(x)
        x = self.act_fn(x)

        x = self.layer2(x)
        x = self.batchNorm2(x)
        x = self.act_fn(x)

        x = self.layer3(x)
        x = self.batchNorm3(x)
        x = self.act_fn(x)
        
        x = self.layer4(x)
        x = self.batchNorm4(x)
        x = self.act_fn(x)
        
        x = self.layer5(x)
        x = self.batchNorm5(x)
        x = self.act_fn(x)
        
        # TODO? 輸出層可以改成softmax看看
        # 變成多一層softmax
        x = self.out(x)
        
        return x

## **Training**

In [ ]:
#check device
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

Fix random seeds for reproducibility.

In [ ]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

Feel free to change the training parameters here.

In [ ]:
# fix random seed for reproducibility
same_seeds(0)

# get device 
device = get_device()
print(f'DEVICE: {device}')

# training parameters
num_epoch = 20               # number of training epoch
learning_rate = 0.0001       # learning rate
weight_decay = 1e-5

# the path where checkpoint saved
model_path = './model.ckpt'

# create model, define a loss function, and optimizer
model = Classifier().to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [ ]:
# start training

best_acc = 0.0
# best_loss = 100000.0 #不是應該用validation的loss來看嗎?? 好像沒有比較好
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(train_loader):
        inputs, labels = data # 包括 11x39 和 label
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() 
        outputs = model(inputs) 
        # TODO? 為什麼不用 crossentropy
        # 助教是用 crossentropy沒錯
        batch_loss = criterion(outputs, labels) 
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward() 
        optimizer.step() 

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels) 
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch    
            
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
            '''
            if val_loss < best_loss:
                best_loss = val_loss
                torch.save(model.state_dict(), model_path)
                print('saving model with loss {:.6f}'.format(best_loss/len(val_loader)))   
            '''
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')

## **Testing**

Create a testing dataset, and load model from the saved checkpoint.

In [ ]:
# create testing dataset
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# create model and load weights from checkpoint
model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

Make prediction.

In [ ]:
predict = []
model.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict.append(y)

Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [ ]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))

# **Hints**

## **Simple baseline**
* You should able to pass the simple baseline using the sample code provided.

## **Strong baseline**
* Model architecture (layers? dimension? activation function?)
* Training (batch size? optimizer? learning rate? epoch?)
* Tips (batch norm? dropout? regularization?)